<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing a LangChain and VertexAI Application</h1>

LLM orchestration frameworks such as LangChain provide abstractions that enable users to build powerful applications in a few lines of code. However, the same abstractions can make it difficult to understand what is going on under the hood and to pinpoint the cause of issues.

Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your chain and surfacing problematic "spans" of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build a simple retrieval-augmented generation application over the Arize documentation using LangChain and VertexAI, in particular, using "textembedding-gecko" for embeddings and "chat-bison" for chat,
- Record trace data in OpenInference format,
- Inspect the traces and spans of your application to identify uncaught exceptions and sources of latency and cost.

ℹ️️ This notebook requires access to the [Vertex AI API](https://cloud.google.com/vertex-ai/docs).

## 1. Install Dependencies and Import Libraries

Downgrade Colab's pre-installed version of `shapely` for compatibility reasons.

⚠️ If you run into a version compatibility error in a later cell, try restarting the runtime and re-running the notebook.

In [ ]:
!pip install --force-reinstall "shapely<2.0.0"

Install Phoenix, LangChain, and the Google Cloud AI Platform SDK.

In [ ]:
!pip install "arize-phoenix[evals]" google-api-python-client "google-cloud-aiplatform[preview]" langchain nest-asyncio

Import libraries.

In [ ]:
import json
from urllib.request import urlopen

import nest_asyncio
import numpy as np
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.retrievers import KNNRetriever
from phoenix.evals import (
    HallucinationEvaluator,
    QAEvaluator,
    RelevanceEvaluator,
    VertexAIModel,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from phoenix.trace.langchain import LangChainInstrumentor
from tqdm import tqdm

nest_asyncio.apply()  # needed for concurrent evals in notebook environments

try:
    from google.colab.auth import authenticate_user

    IS_COLAB = True
except ImportError:
    IS_COLAB = False

## 2. Set Configuration and Authenticate with Vertex AI

If you are running this notebook in Colab, sign in with your Gmail credentials. If running locally, you'll need to ensure that your `gcloud` is correctly configured to run Vertex AI.

In [ ]:
if IS_COLAB:
    authenticate_user()
else:
    print(
        "If running locally, ensure that your gcloud is correctly configured to run with Vertex AI."
    )

Enter your project ID and location:
* `project_id`: The default project to use when making API calls.
* `location`: The default location to use when making API calls. If not set defaults to us-central-1.

In [ ]:
project_id = input("Enter your GCP project ID and press enter:\n")
location = input("Enter your GCP location and press enter:\n")

## 3. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the `OpenInferenceTraceCallbackHandler`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run our LangChain application).

In [ ]:
session = px.launch_app()

## 4. Instantiate LangChain Instrumentor

In [ ]:
LangChainInstrumentor().instrument()

## 5. Build Your LLM Application

Define a `RetrievalQA` chain leveraging "textembedding-gecko" and "chat-bison" from the VertexAI API. The knowledge base of this chain is built over the Arize documentation.

In [ ]:
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko",
    project=project_id,
    location=location,
)
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain-pinecone-vertexai/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(database_df["text_vector"]),
    texts=database_df["text"].tolist(),
    embeddings=embeddings,
)
llm = ChatVertexAI(
    model_name="chat-bison",
    project=project_id,
    location=location,
)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=knn_retriever,
)

## 6. Run the Chain

Download a small dataset of user queries to ask your application.

In [ ]:
url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]

Run your chain against ten queries. Notice that the tracer is attached in the `run` call.

In [ ]:
for query in tqdm(queries[:10]):
    chain.invoke(query)

Check out your traces in Phoenix!

In [ ]:
print(f"Open the Phoenix UI if you haven't already: {session.url}")

## 7. Export and Evaluate Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In this case, we will export our `retriever` spans into two separate dataframes:
- `queries_df`, in which the retrieved documents for each query are concatenated into a single column,
- `retrieved_documents_df`, in which each retrieved document is "exploded" into its own row to enable the evaluation of each query-document pair in isolation.

This will enable us to compute multiple kinds of evaluations, including:
- relevance: Are the retrieved documents grounded in the response?
- Q&A correctness: Are your application's responses grounded in the retrieved context?
- hallucinations: Is your application making up false information?

In [ ]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

Next, define your evaluation model and your evaluators.

Evaluators are built on top of language models and prompt the LLM to assess the quality of responses, the relevance of retrieved documents, etc., and provide a quality signal even in the absence of human-labeled data. Pick an evaluator type and instantiate it with the language model you want to use to perform evaluations using our battle-tested evaluation templates.

In [ ]:
eval_model = VertexAIModel(
    project=project_id,
    location=location,
    model_name="text-bison",
    temperature=0.0,
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

Your evaluations should now appear as annotations on the appropriate spans in Phoenix.

In [ ]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")

## 8. Final Thoughts

LLM Traces and the accompanying OpenInference Tracing specification is designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context such as retrieval from vector stores and the usage of external tools such as search engines or APIs. It lets you understand the inner workings of the individual steps your application takes wile also giving you visibility into how your system is running and performing as a whole. 

LLM Evals are designed for simple, fast, and accurate LLM-based evaluations. They let you quickly benchmark the performance of your LLM application and help you identify the problematic spans of execution.

For more details on Phoenix, LLM Tracing, and LLM Evals, checkout our [documentation](https://docs.arize.com/phoenix/).